#ESRGAN+ (2D)
---


# **How to use this notebook?**

---

<font size = 4>Video describing how to use ZeroCostDL4Mic notebooks are available on youtube:
  - [**Video 1**](https://www.youtube.com/watch?v=GzD2gamVNHI&feature=youtu.be): Full run through of the workflow to obtain the notebooks and the provided test datasets as well as a common use of the notebook
  - [**Video 2**](https://www.youtube.com/watch?v=PUuQfP5SsqM&feature=youtu.be): Detailed description of the different sections of the notebook


---
###**Structure of a notebook**

<font size = 4>The notebook contains two types of cell:  

<font size = 4>**Text cells** provide information and can be modified by douple-clicking the cell. You are currently reading the text cell. You can create a new text by clicking `+ Text`.

<font size = 4>**Code cells** contain code and the code can be modfied by selecting the cell. To execute the cell, move your cursor on the `[ ]`-mark on the left side of the cell (play button appears). Click to execute the cell. After execution is done the animation of play button stops. You can create a new coding cell by clicking `+ Code`.

---
###**Table of contents, Code snippets** and **Files**

<font size = 4>On the top left side of the notebook you find three tabs which contain from top to bottom:

<font size = 4>*Table of contents* = contains structure of the notebook. Click the content to move quickly between sections.

<font size = 4>*Code snippets* = contain examples how to code certain tasks. You can ignore this when using this notebook.

<font size = 4>*Files* = contain all available files. After mounting your google drive (see section 1.) you will find your files and folders here. 

<font size = 4>**Remember that all uploaded files are purged after changing the runtime.** All files saved in Google Drive will remain. You do not need to use the Mount Drive-button; your Google Drive is connected in section 1.2.

<font size = 4>**Note:** The "sample data" in "Files" contains default files. Do not upload anything in here!

---
###**Making changes to the notebook**

<font size = 4>**You can make a copy** of the notebook and save it to your Google Drive. To do this click file -> save a copy in drive.

<font size = 4>To **edit a cell**, double click on the text. This will show you either the source code (in code cells) or the source text (in text cells).
You can use the `#`-mark in code cells to comment out parts of the code. This allows you to keep the original code piece in the cell as a comment.

#**0. Before getting started**
---

<font size = 4>**We strongly recommend that you generate extra paired images. These images can be used to assess the quality of your trained model (Quality control dataset)**. The quality control assessment can be done directly in this notebook.

<font size = 4> **Additionally, the corresponding input and output files need to have the same name**.

<font size = 4> Please note that you currently can **only use .tif files!**


<font size = 4>Here's a common data structure that can work:
*   Experiment A
    - **Training dataset**
      - Low resolution (LR) images (Training_source)
        - img_1.tif, img_2.tif, ...
      - High resolution (HR) images (Training_target)
        - img_1.tif, img_2.tif, ...
    - **Quality control dataset**
     - Low resolution (LR) images
        - img_1.tif, img_2.tif
      - High resolution (HR) images
        - img_1.tif, img_2.tif
    - **Data to be predicted**
    - **Results**

---
<font size = 4>**Important note**

<font size = 4>- If you wish to **Train a network from scratch** using your own dataset (and we encourage everyone to do that), you will need to run **sections 1 - 4**, then use **section 5** to assess the quality of your model and **section 6** to run predictions using the model that you trained.

<font size = 4>- If you wish to **Evaluate your model** using a model previously generated and saved on your Google Drive, you will only need to run **sections 1 and 2** to set up the notebook, then use **section 5** to assess the quality of your model.

<font size = 4>- If you only wish to **run predictions** using a model previously generated and saved on your Google Drive, you will only need to run **sections 1 and 2** to set up the notebook, then use **section 6** to run the predictions on the desired model.
---

# **1. Install ESRGAN+ and dependencies**
---

In [ ]:
#@markdown ##Install Network and dependencies
Notebook_version = '1.13'
Network = 'ESRGANplus (2D)'

!pip install pytorch-lightning --quiet

import sys
sys.setrecursionlimit(10000)

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torchvision
import random
import shutil
import math
import csv
import os
import functools
import logging
logger = logging.getLogger('base')

from collections import defaultdict, OrderedDict
from tqdm import tqdm

import skimage
from skimage import transform
from skimage import filters
from skimage import metrics
from skimage import io
from skimage.util import random_noise
from skimage.util import img_as_ubyte, img_as_uint, img_as_float32
from skimage.metrics import structural_similarity, peak_signal_noise_ratio

import torch
from torch import nn
from torch.nn import init
from torch.utils.data import Dataset, DataLoader

from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core import LightningModule
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.loggers import CSVLogger

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tensorflow.image import ssim_multiscale as mssim

from ipywidgets import interact, interactive, fixed, interact_manual
from scipy.ndimage.interpolation import zoom as npzoom
from torchvision.utils import save_image

###

def set_seed(seed=56):
    ''' Sets the seed for the random operation in all 
        the packagees that use it.
    Args:
        seed (int): seed that will be established.
    '''
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed()

###

def em_crappify(img, scale):
    ''' Degradates and downscales the given image, simuating
        the degradation that a electron microscopy would suffer.
        Source https://www.biorxiv.org/content/10.1101/740548v3.
    Args:
        img (array): image that whose low resolution counterpart will be calcualted.
        scale (int): scale factor for the image downscaling. Example: 2.
    Returns:
        Low resolution image obtained from degradatin the received high resolution image.
    '''
    img = filters.gaussian(img, sigma=3) + 1e-6
    return transform.resize(img, (img.shape[0]//scale, img.shape[1]//scale), order=1)

def fluo_crappify(img,scale):
    ''' Degradates and downscales the given image, simuating
        the degradation that a fluorescence microscopy would suffer.
        Source https://www.biorxiv.org/content/10.1101/740548v3.
    Args:
        img (array): image that whose low resolution counterpart will be calcualted.
        scale (int): scale factor for the image downscaling. Example: 2.
    Returns:
        Low resolution image obtained from degradatin the received high resolution image.
    '''
    img = random_noise(img, mode='salt', amount=0.005)
    img = random_noise(img, mode='pepper', amount=0.005)
    img = filters.gaussian(img, sigma=5) + 1e-10
    return transform.resize(img, (img.shape[0]//scale, img.shape[1]//scale), order=1)  
###

def calculate_down_factor (lr_imgs_basedir, hr_imgs_basedir):
    _, hr_extension = os.path.splitext(os.listdir(hr_imgs_basedir)[0])
    hr_filenames = [hr_imgs_basedir + '/' + x for x in os.listdir(hr_imgs_basedir) if x.endswith(hr_extension)]

    _, lr_extension = os.path.splitext(os.listdir(lr_imgs_basedir)[0])
    lr_filenames = [lr_imgs_basedir + '/' + x for x in os.listdir(lr_imgs_basedir) if x.endswith(lr_extension)]

    hr_img = io.imread(hr_filenames[0])
    lr_img = io.imread(lr_filenames[0])

    down_scale_x = hr_img.shape[0] / lr_img.shape[0]
    down_scale_y = hr_img.shape[1] / lr_img.shape[1]

    return [down_scale_x, down_scale_y]

###

class bcolors:
    '''Definition of a color for the displays.'''
    WARNING = '\033[31m'

class ToTensor(object):
    '''Convert ndarrays in sample to Tensors.'''
    def __call__(self, sample):
        hr, lr = sample['hr'], sample['lr']

        # Pytorch is (batch, channels, width, height)
        hr = hr.transpose((2, 0, 1))
        lr = lr.transpose((2, 0, 1))
        return {'hr': torch.from_numpy(hr),
                'lr': torch.from_numpy(lr)}

class RandomHorizontalFlip(object):
    '''Random horizontal flip.'''
    def __init__(self):
        self.rng = np.random.default_rng()

    def __call__(self, sample):
        hr, lr = sample['hr'], sample['lr']

        if self.rng.random() < 0.5:
            hr = np.flip(hr, 1)
            lr = np.flip(lr, 1)

        return {'hr': hr.copy(),
                'lr': lr.copy()}

class RandomVerticalFlip(object):
    '''Random vertical flip.'''
    def __init__(self):
        self.rng = np.random.default_rng()

    def __call__(self, sample):
        hr, lr = sample['hr'], sample['lr']

        if self.rng.random() < 0.5:
            hr = np.flip(hr, 0)
            lr = np.flip(lr, 0)

        return {'hr': hr.copy(),
                'lr': lr.copy()}

class RandomRotate(object):
    '''Random rotation.'''
    def __init__(self):
        self.rng = np.random.default_rng()

    def __call__(self, sample):
        hr, lr = sample['hr'], sample['lr']

        k = self.rng.integers(4)

        hr = np.rot90(hr, k=k)
        lr = np.rot90(lr, k=k)

        return {'hr': hr.copy(),
                'lr': lr.copy()}

class EMDataset(Dataset):
    ''' Pytorch's Dataset type object used to obtain the train and 
        validation information during the training process. Saves the 
        filenames as an attribute and only loads the ones rquired for
        the training batch, reducing the required RAM memory during 
        and after the training.
    '''
    def __init__(self, 
                 patch_size_x, 
                 patch_size_y,
                 down_factor,
                 transf=None, 
                 validation=False, 
                 validation_split=None,
                 hr_imgs_basedir="", 
                 lr_imgs_basedir="",
                 only_high_resolution_data=False,
                 only_hr_imgs_basedir="",
                 type_of_data="Electron microscopy"):

        if only_high_resolution_data:
            used_hr_imgs_basedir = only_hr_imgs_basedir 
        else: 
            used_hr_imgs_basedir = hr_imgs_basedir

        _, hr_extension = os.path.splitext(os.listdir(used_hr_imgs_basedir)[0])

        hr_filenames = [used_hr_imgs_basedir + '/' + x for x in os.listdir(used_hr_imgs_basedir) if x.endswith(hr_extension)]
        hr_filenames.sort()

        if validation_split is not None:
            val_files = int(len(hr_filenames) * validation_split)
            if validation:
                self.hr_img_names = hr_filenames[:val_files]
            else:
                self.hr_img_names = hr_filenames[val_files:]
        else:
            self.hr_img_names = hr_filenames

        if not only_high_resolution_data:
            _, lr_extension = os.path.splitext(os.listdir(lr_imgs_basedir)[0])

            lr_filenames = [lr_imgs_basedir + '/' + x for x in os.listdir(lr_imgs_basedir) if x.endswith(lr_extension)]
            lr_filenames.sort()

            if validation_split is not None:
                val_lr_files = int(len(lr_filenames) * validation_split)
                if validation:
                    self.lr_img_names = lr_filenames[:val_lr_files]
                else:
                    self.lr_img_names = lr_filenames[val_lr_files:]
            else:
                self.lr_img_names = lr_filenames

        self.only_high_resolution_data = only_high_resolution_data

        self.transf = transf
        self.down_factor = down_factor
        self.type_of_data = type_of_data

        self.lr_patch_size_x = patch_size_x
        self.lr_patch_size_y = patch_size_y
        self.hr_patch_size_x = patch_size_x * down_factor
        self.hr_patch_size_y = patch_size_y * down_factor

        self.lr_shape = io.imread(self.hr_img_names[0]).shape[0]//down_factor

    def hr_to_lr(self, hr_img, down_factor, type_of_data):

        if type_of_data == "Electron microscopy":
            lr_img = em_crappify(hr_img, down_factor)
        else:
            lr_img = fluo_crappify(hr_img, down_factor)
        return lr_img

    def __len__(self):
        return len(self.hr_img_names) * (self.lr_shape//self.lr_patch_size_x)**2

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_idx = idx // (self.lr_shape//self.lr_patch_size_x)**2
        
        hr_img = img_as_float32(io.imread(self.hr_img_names[img_idx]))
        
        if self.only_high_resolution_data:
            lr_img = self.hr_to_lr(hr_img, self.down_factor, self.type_of_data)
        else:
            lr_img = img_as_float32(io.imread(self.lr_img_names[img_idx]))

        lr_idx_x = np.random.randint(0, max(1, lr_img.shape[0] - self.lr_patch_size_x))
        lr_idx_y = np.random.randint(0, max(1, lr_img.shape[1] - self.lr_patch_size_y))

        lr_patch = lr_img[lr_idx_x : lr_idx_x + self.lr_patch_size_x, 
                          lr_idx_y : lr_idx_y + self.lr_patch_size_y]
        lr_patch = lr_patch[:,:,np.newaxis]

        hr_idx_x = lr_idx_x * self.down_factor
        hr_idx_y = lr_idx_y * self.down_factor

        hr_patch = hr_img[hr_idx_x : hr_idx_x + self.hr_patch_size_x, 
                          hr_idx_y : hr_idx_y + self.hr_patch_size_x]

        hr_patch = hr_patch[:,:,np.newaxis]

        sample = {'hr': hr_patch, 'lr': lr_patch}

        if self.transf:
            sample = self.transf(sample)

        return sample

####################
# Basic blocks
####################


def act(act_type, inplace=True, neg_slope=0.2, n_prelu=1):
    # helper selecting activation
    # neg_slope: for leakyrelu and init of prelu
    # n_prelu: for p_relu num_parameters
    act_type = act_type.lower()
    if act_type == 'relu':
        layer = nn.ReLU(inplace)
    elif act_type == 'leakyrelu':
        layer = nn.LeakyReLU(neg_slope, inplace)
    elif act_type == 'prelu':
        layer = nn.PReLU(num_parameters=n_prelu, init=neg_slope)
    else:
        raise NotImplementedError('activation layer [{:s}] is not found'.format(act_type))
    return layer


def norm(norm_type, nc):
    # helper selecting normalization layer
    norm_type = norm_type.lower()
    if norm_type == 'batch':
        layer = nn.BatchNorm2d(nc, affine=True)
    elif norm_type == 'instance':
        layer = nn.InstanceNorm2d(nc, affine=False)
    else:
        raise NotImplementedError('normalization layer [{:s}] is not found'.format(norm_type))
    return layer


def pad(pad_type, padding):
    # helper selecting padding layer
    # if padding is 'zero', do by conv layers
    pad_type = pad_type.lower()
    if padding == 0:
        return None
    if pad_type == 'reflect':
        layer = nn.ReflectionPad2d(padding)
    elif pad_type == 'replicate':
        layer = nn.ReplicationPad2d(padding)
    else:
        raise NotImplementedError('padding layer [{:s}] is not implemented'.format(pad_type))
    return layer


def get_valid_padding(kernel_size, dilation):
    kernel_size = kernel_size + (kernel_size - 1) * (dilation - 1)
    padding = (kernel_size - 1) // 2
    return padding


class ShortcutBlock(nn.Module):
    #Elementwise sum the output of a submodule to its input
    def __init__(self, submodule):
        super(ShortcutBlock, self).__init__()
        self.sub = submodule

    def forward(self, x):
        output = x + self.sub(x)
        return output

    def __repr__(self):
        tmpstr = 'Identity + \n|'
        modstr = self.sub.__repr__().replace('\n', '\n|')
        tmpstr = tmpstr + modstr
        return tmpstr


def sequential(*args):
    # Flatten Sequential. It unwraps nn.Sequential.
    if len(args) == 1:
        if isinstance(args[0], OrderedDict):
            raise NotImplementedError('sequential does not support OrderedDict input.')
        return args[0]  # No sequential is needed.
    modules = []
    for module in args:
        if isinstance(module, nn.Sequential):
            for submodule in module.children():
                modules.append(submodule)
        elif isinstance(module, nn.Module):
            modules.append(module)
    return nn.Sequential(*modules)

class GaussianNoise(nn.Module):
    def __init__(self, sigma=0.1, is_relative_detach=False):
        super().__init__()
        self.sigma = sigma
        self.is_relative_detach = is_relative_detach
        self.noise = torch.tensor(0, dtype=torch.float).to(torch.device('cuda'))

    def forward(self, x):
        if self.training and self.sigma != 0:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            sampled_noise = self.noise.repeat(*x.size()).normal_() * scale
            x = x + sampled_noise
        return x 


def conv_block(in_nc, out_nc, kernel_size, stride=1, dilation=1, groups=1, bias=True, \
               pad_type='zero', norm_type=None, act_type='relu', mode='CNA'):
    '''
    Conv layer with padding, normalization, activation
    mode: CNA --> Conv -> Norm -> Act
        NAC --> Norm -> Act --> Conv (Identity Mappings in Deep Residual Networks, ECCV16)
    '''
    assert mode in ['CNA', 'NAC', 'CNAC'], 'Wong conv mode [{:s}]'.format(mode)
    padding = get_valid_padding(kernel_size, dilation)
    p = pad(pad_type, padding) if pad_type and pad_type != 'zero' else None
    padding = padding if pad_type == 'zero' else 0

    c = nn.Conv2d(in_nc, out_nc, kernel_size=kernel_size, stride=stride, padding=padding, \
            dilation=dilation, bias=bias, groups=groups)
    a = act(act_type) if act_type else None
    if 'CNA' in mode:
        n = norm(norm_type, out_nc) if norm_type else None
        return sequential(p, c, n, a)
    elif mode == 'NAC':
        if norm_type is None and act_type is not None:
            a = act(act_type, inplace=False)
            # Important!
            # input----ReLU(inplace)----Conv--+----output
            #        |________________________|
            # inplace ReLU will modify the input, therefore wrong output
        n = norm(norm_type, in_nc) if norm_type else None
        return sequential(n, a, p, c)
  
def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


####################
# Useful blocks
####################


class ResidualDenseBlock_5C(nn.Module):
    '''
    Residual Dense Block
    style: 5 convs
    The core module of paper: (Residual Dense Network for Image Super-Resolution, CVPR 18)
    '''

    def __init__(self, nc, kernel_size=3, gc=32, stride=1, bias=True, pad_type='zero', \
            norm_type=None, act_type='leakyrelu', mode='CNA', gaussian_noise=True):
        super(ResidualDenseBlock_5C, self).__init__()
        # gc: growth channel, i.e. intermediate channels
        self.noise = GaussianNoise() if gaussian_noise else None
        self.conv1x1 = conv1x1(nc, gc)
        self.conv1 = conv_block(nc, gc, kernel_size, stride, bias=bias, pad_type=pad_type, \
            norm_type=norm_type, act_type=act_type, mode=mode)
        self.conv2 = conv_block(nc+gc, gc, kernel_size, stride, bias=bias, pad_type=pad_type, \
            norm_type=norm_type, act_type=act_type, mode=mode)
        self.conv3 = conv_block(nc+2*gc, gc, kernel_size, stride, bias=bias, pad_type=pad_type, \
            norm_type=norm_type, act_type=act_type, mode=mode)
        self.conv4 = conv_block(nc+3*gc, gc, kernel_size, stride, bias=bias, pad_type=pad_type, \
            norm_type=norm_type, act_type=act_type, mode=mode)
        if mode == 'CNA':
            last_act = None
        else:
            last_act = act_type
        self.conv5 = conv_block(nc+4*gc, nc, 3, stride, bias=bias, pad_type=pad_type, \
            norm_type=norm_type, act_type=last_act, mode=mode)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(torch.cat((x, x1), 1))
        x2 = x2 + self.conv1x1(x)
        x3 = self.conv3(torch.cat((x, x1, x2), 1))
        x4 = self.conv4(torch.cat((x, x1, x2, x3), 1))
        x4 = x4 + x2
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        return self.noise(x5.mul(0.2) + x)


class RRDB(nn.Module):
    '''
    Residual in Residual Dense Block
    (ESRGAN: Enhanced Super-Resolution Generative Adversarial Networks)
    '''

    def __init__(self, nc, kernel_size=3, gc=32, stride=1, bias=True, pad_type='zero', \
            norm_type=None, act_type='leakyrelu', mode='CNA'):
        super(RRDB, self).__init__()
        self.RDB1 = ResidualDenseBlock_5C(nc, kernel_size, gc, stride, bias, pad_type, \
            norm_type, act_type, mode)
        self.RDB2 = ResidualDenseBlock_5C(nc, kernel_size, gc, stride, bias, pad_type, \
            norm_type, act_type, mode)
        self.RDB3 = ResidualDenseBlock_5C(nc, kernel_size, gc, stride, bias, pad_type, \
            norm_type, act_type, mode)

    def forward(self, x):
        out = self.RDB1(x)
        out = self.RDB2(out)
        out = self.RDB3(out)
        return out.mul(0.2) + x


####################
# Upsampler
####################


def pixelshuffle_block(in_nc, out_nc, upscale_factor=2, kernel_size=3, stride=1, bias=True, \
                        pad_type='zero', norm_type=None, act_type='relu'):
    '''
    Pixel shuffle layer
    (Real-Time Single Image and Video Super-Resolution Using an Efficient Sub-Pixel Convolutional
    Neural Network, CVPR17)
    '''
    conv = conv_block(in_nc, out_nc * (upscale_factor ** 2), kernel_size, stride, bias=bias, \
                        pad_type=pad_type, norm_type=None, act_type=None)
    pixel_shuffle = nn.PixelShuffle(upscale_factor)

    n = norm(norm_type, out_nc) if norm_type else None
    a = act(act_type) if act_type else None
    return sequential(conv, pixel_shuffle, n, a)


def upconv_blcok(in_nc, out_nc, upscale_factor=2, kernel_size=3, stride=1, bias=True, \
                pad_type='zero', norm_type=None, act_type='relu', mode='nearest'):
    # Up conv
    # described in https://distill.pub/2016/deconv-checkerboard/
    upsample = nn.Upsample(scale_factor=upscale_factor, mode=mode)
    conv = conv_block(in_nc, out_nc, kernel_size, stride, bias=bias, \
                        pad_type=pad_type, norm_type=norm_type, act_type=act_type)
    return sequential(upsample, conv)


class RRDBNet(nn.Module):
    def __init__(self, in_nc, out_nc, nf, nb, gc=32, upscale=4, norm_type=None, \
            act_type='leakyrelu', mode='CNA', upsample_mode='upconv'):
        super(RRDBNet, self).__init__()
        n_upscale = int(math.log(upscale, 2))
        if upscale == 3:
            n_upscale = 1

        fea_conv = conv_block(in_nc, nf, kernel_size=3, norm_type=None, act_type=None)
        rb_blocks = [RRDB(nf, kernel_size=3, gc=32, stride=1, bias=True, pad_type='zero', \
            norm_type=norm_type, act_type=act_type, mode='CNA') for _ in range(nb)]
        LR_conv = conv_block(nf, nf, kernel_size=3, norm_type=norm_type, act_type=None, mode=mode)

        if upsample_mode == 'upconv':
            upsample_block = upconv_blcok
        elif upsample_mode == 'pixelshuffle':
            upsample_block = pixelshuffle_block
        else:
            raise NotImplementedError('upsample mode [{:s}] is not found'.format(upsample_mode))
        if upscale == 3:
            upsampler = upsample_block(nf, nf, 3, act_type=act_type)
        else:
            upsampler = [upsample_block(nf, nf, act_type=act_type) for _ in range(n_upscale)]
        HR_conv0 = conv_block(nf, nf, kernel_size=3, norm_type=None, act_type=act_type)
        HR_conv1 = conv_block(nf, out_nc, kernel_size=3, norm_type=None, act_type=None)

        self.model = sequential(fea_conv, ShortcutBlock(sequential(*rb_blocks, LR_conv)),\
            *upsampler, HR_conv0, HR_conv1)

    def forward(self, x):
        x = self.model(x)
        return x


# VGG style Discriminator with input size 128*128
class Discriminator_VGG_128(nn.Module):
    def __init__(self, in_nc, base_nf, norm_type='batch', act_type='leakyrelu', mode='CNA'):
        super(Discriminator_VGG_128, self).__init__()
        # features
        # hxw, c
        # 128, 64
        conv0 = conv_block(in_nc, base_nf, kernel_size=3, norm_type=None, act_type=act_type, \
            mode=mode)
        conv1 = conv_block(base_nf, base_nf, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 64, 64
        conv2 = conv_block(base_nf, base_nf*2, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv3 = conv_block(base_nf*2, base_nf*2, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 32, 128
        conv4 = conv_block(base_nf*2, base_nf*4, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv5 = conv_block(base_nf*4, base_nf*4, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 16, 256
        conv6 = conv_block(base_nf*4, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv7 = conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 8, 512
        conv8 = conv_block(base_nf*8, base_nf*8, kernel_size=3, stride=1, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        conv9 = conv_block(base_nf*8, base_nf*8, kernel_size=4, stride=2, norm_type=norm_type, \
            act_type=act_type, mode=mode)
        # 4, 512
        self.features = sequential(conv0, conv1, conv2, conv3, conv4, conv5, conv6, conv7, conv8,\
            conv9)

        # classifier
        self.classifier = nn.Sequential(
            nn.Linear(512 * 16 * 16, 100), nn.LeakyReLU(0.2, True), nn.Linear(100, 1))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


# Assume input range is [0, 1]
class VGGFeatureExtractor(nn.Module):
    def __init__(self,
                 feature_layer=34,
                 use_bn=False,
                 use_input_norm=True,
                 device=torch.device('cpu')):
        super(VGGFeatureExtractor, self).__init__()
        if use_bn:
            model = torchvision.models.vgg19_bn(pretrained=True)
        else:
            model = torchvision.models.vgg19(pretrained=True)
        self.use_input_norm = use_input_norm
        if self.use_input_norm:
            mean = torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(device)
            # [0.485-1, 0.456-1, 0.406-1] if input in range [-1,1]
            std = torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(device)
            # [0.229*2, 0.224*2, 0.225*2] if input in range [-1,1]
            self.register_buffer('mean', mean)
            self.register_buffer('std', std)
        self.features = nn.Sequential(*list(model.features.children())[:(feature_layer + 1)])
        # No need to BP to variable
        for k, v in self.features.named_parameters():
            v.requires_grad = False

    def forward(self, x):
        if self.use_input_norm:
            x = (x - self.mean) / self.std
        output = self.features(x)
        return output


# Define GAN loss
class GANLoss(nn.Module):
    def __init__(self, real_label_val=1.0, fake_label_val=0.0):
        super(GANLoss, self).__init__()
        self.real_label_val = real_label_val
        self.fake_label_val = fake_label_val

        self.loss = nn.BCEWithLogitsLoss()

    def get_target_label(self, input, target_is_real):
        if target_is_real:
            return torch.empty_like(input).fill_(self.real_label_val)
        else:
            return torch.empty_like(input).fill_(self.fake_label_val)

    def forward(self, input, target_is_real):
        target_label = self.get_target_label(input, target_is_real)
        loss = self.loss(input, target_label)
        return loss

def weights_init_kaiming(m, scale=1):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
        m.weight.data *= scale
        if m.bias is not None:
            m.bias.data.zero_()
    elif classname.find('Linear') != -1:
        init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
        m.weight.data *= scale
        if m.bias is not None:
            m.bias.data.zero_()
    elif classname.find('BatchNorm2d') != -1:
        init.constant_(m.weight.data, 1.0)
        init.constant_(m.bias.data, 0.0)

# Generator
def define_G(scale):
    
    netG = RRDBNet(in_nc=1, out_nc=1, nf=64, nb=23, gc=32, upscale=scale, 
                   norm_type=None, act_type='leakyrelu', mode="CNA",
                   upsample_mode='upconv')

    weights_init_kaiming_ = functools.partial(weights_init_kaiming, scale=0.1)
    netG.apply(weights_init_kaiming_)
    
    if torch.cuda.is_available():
        netG = nn.DataParallel(netG)
    return netG


# Discriminator
def define_D():

    netD = Discriminator_VGG_128(in_nc=1, base_nf=64, norm_type="batch", 
                                 mode="CNA", act_type="leakyrelu")

    weights_init_kaiming_ = functools.partial(weights_init_kaiming, scale=0.1)
    netD.apply(weights_init_kaiming_)
    
    if torch.cuda.is_available():
        netD = nn.DataParallel(netD)
    return netD

def define_F(use_bn=False):
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    if use_bn:
        feature_layer = 49
    else:
        feature_layer = 34
    netF = VGGFeatureExtractor(feature_layer=feature_layer, use_bn=use_bn,
                               use_input_norm=True, device=device)
        
    if torch.cuda.is_available():
        netF = nn.DataParallel(netF)
    netF.eval()  # No need to train
    return netF

###

class ESRGANplus(LightningModule):
    def __init__(self,
               batchsize: int = 8,
               patch_size: int = 64,
               down_factor: int = 2,
               learning_rate_d: float = 0.0001,
               learning_rate_g: float = 0.0001,
               n_critic_steps: int = 5,
               validation_split: float = 0.1,
               epochs: int = 151,
               rotation: bool = True,
               horizontal_flip: bool = True,
               vertical_flip: bool = True,
               hr_imgs_basedir: str = "", 
               lr_imgs_basedir: str ="",
               only_high_resolution_data: bool = False,
               only_hr_images_basedir: str = "",
               type_of_data: str = "Electron microscopy",
               gen_checkpoint: str = None, 
               save_basedir: str = None,
               only_predict: bool = False
               ):
        super(ESRGANplus, self).__init__()
        
        self.save_hyperparameters()

        if gen_checkpoint is not None:
            checkpoint = torch.load(gen_checkpoint)
            self.generator = define_G(checkpoint['down_factor'])
            self.generator.load_state_dict(checkpoint['model_state_dict'])
            self.best_valid_loss = checkpoint['best_valid_loss']
        else:
            self.generator = define_G(down_factor)
            self.best_valid_loss = float('inf')

        self.discriminator = define_D()
        
        # Generator ixel loss
        self.cri_pix = nn.L1Loss()
        self.l_pix_w = 0.01

        # G feature loss
        self.cri_fea = nn.L1Loss()
        self.l_fea_w = 1

        # load VGG perceptual loss
        self.netF = define_F(use_bn=False)

        # GD gan loss
        self.cri_gan = GANLoss(1.0, 0.0)
        self.l_gan_w = 0.005

        # D_update_ratio and D_init_iters are for WGAN
        self.D_update_ratio = 1
        self.D_init_iters = 0

        self.optimizer_G = None
        self.optimizer_D = None

        if not only_predict:
            self.len_data = self.train_dataloader().__len__()
            self.total_iters = epochs * self.len_data
            
    def save_model(self, filename):
        if self.hparams.save_basedir is not None:
            torch.save({
                        'model_state_dict': self.generator.state_dict(),
                        'optimizer_state_dict': self.optimizer_G.state_dict(),
                        'down_factor': self.hparams.down_factor,
                        'best_valid_loss': self.best_valid_loss
                        }, self.hparams.save_basedir + '/' + filename)
        else:
            raise Exception('No save_basedir was specified in the construction of the WGAN object.')

    def forward(self, x):
        return self.generator(x)
  
    def training_step(self, batch, batch_idx, optimizer_idx):
        lr, hr = batch['lr'], batch['hr']
        fake_hr = self.generator(lr)

        # Optimize generator
        if optimizer_idx == 0:
            l_g_total = 0

            # pixel loss
            l_g_pix = self.l_pix_w * self.cri_pix(fake_hr, hr)
            l_g_total += l_g_pix
            
            # feature loss
            real_fea = self.netF(hr)
            fake_fea = self.netF(fake_hr)
            l_g_fea = self.l_fea_w * self.cri_fea(fake_fea, real_fea)
            l_g_total += l_g_fea
            
            # G gan + cls loss
            pred_g_fake = self.discriminator(fake_hr)
            #pred_d_real = self.discriminator(hr).detach()
            pred_d_real = self.discriminator(hr)

            l_g_gan = self.l_gan_w * (self.cri_gan(pred_d_real - torch.mean(pred_g_fake), False) +
                                      self.cri_gan(pred_g_fake - torch.mean(pred_d_real), True)) / 2
            l_g_total += l_g_gan

            self.log('g_loss', l_g_total, prog_bar=True, on_epoch=True)
            self.log('g_pixel_loss', l_g_pix, prog_bar=True, on_epoch=True)
            self.log('g_features_loss', l_g_pix, prog_bar=True, on_epoch=True)
            self.log('g_adversarial_loss', l_g_gan, prog_bar=True, on_epoch=True)

            return l_g_total

        # Optimize discriminator
        elif optimizer_idx == 1:
            l_d_total = 0
            pred_d_real = self.discriminator(hr)
            pred_d_fake = self.discriminator(fake_hr)  # detach to avoid BP to G
            l_d_real = self.cri_gan(pred_d_real - torch.mean(pred_d_fake), True)
            l_d_fake = self.cri_gan(pred_d_fake - torch.mean(pred_d_real), False)

            l_d_total = (l_d_real + l_d_fake) / 2

            self.log('d_loss', l_d_total, prog_bar=True, on_epoch=True)
            self.log('d_real', l_d_real, prog_bar=False, on_epoch=True)
            self.log('d_fake', l_d_fake, prog_bar=False, on_epoch=True)

            return l_d_total

    def configure_optimizers(self):

        # G
        n_critic = self.hparams.n_critic_steps

        optim_params = []
        for k, v in self.generator.named_parameters():  # can optimize for a part of the model
            if v.requires_grad:
                optim_params.append(v)

        if self.hparams.gen_checkpoint is not None:
            self.optimizer_G = torch.optim.Adam(optim_params, 
                                    lr=self.hparams.learning_rate_g,
                                    weight_decay=0, betas=(0.9, 0.999))
    
            checkpoint = torch.load(self.hparams.gen_checkpoint)
            self.optimizer_G.load_state_dict(checkpoint['optimizer_state_dict'])
        else:
            self.optimizer_G = torch.optim.Adam(optim_params, 
                                    lr=self.hparams.learning_rate_g,
                                    weight_decay=0, betas=(0.9, 0.999))
        
        # D
        self.optimizer_D = torch.optim.Adam(self.discriminator.parameters(), 
                                            lr=self.hparams.learning_rate_d, 
                                            weight_decay=0, betas=(0.9, 0.999))
        
        sched_g = {
            'scheduler': torch.optim.lr_scheduler.MultiStepLR(
                self.optimizer_G, 
                milestones=[50000,100000,200000,300000], 
                gamma=0.5
            ),
            'interval': 'step',
            'name': 'g_lr'
        }
        sched_d = {
            'scheduler': torch.optim.lr_scheduler.MultiStepLR(
                self.optimizer_D,
                milestones=[50000,100000,200000,300000], 
                gamma=0.5
            ),
            'interval': 'step',
            'name': 'd_lr'
        }

        return (
            {'optimizer': self.optimizer_G, 'frequency': 1, 'lr_scheduler': sched_g},
            {'optimizer': self.optimizer_D, 'frequency': n_critic, 'lr_scheduler': sched_d}
        )

    def validation_step(self, batch, batch_idx):
        # Right now used for just plotting, might want to change it later
        lr, hr = batch['lr'], batch['hr']

        fake_hr = self.generator(lr)

        true = hr.cpu().numpy()
        fake = fake_hr.cpu().numpy()
        
        for i in range(lr.size(0)):
            ssim = structural_similarity(true[i,0,...], 
                                         fake[i,0,...], data_range=1.0)
            self.log('val_ssim', ssim)
            
            psnr = peak_signal_noise_ratio(true[i,0,...], 
                                           fake[i,0,...], data_range=1.0)
            self.log('val_psnr', psnr)
            
        return lr, hr, fake_hr
       
    def validation_step_end(self, val_step_outputs):
        # Right now used for just plotting, might want to change it later
        lr, hr, fake_hr = val_step_outputs
        
        l_g_total = 0

        # pixel loss
        l_g_pix = self.l_pix_w * self.cri_pix(fake_hr, hr)
        l_g_total += l_g_pix
        
        # feature loss
        real_fea = self.netF(hr).detach()
        fake_fea = self.netF(fake_hr)
        l_g_fea = self.l_fea_w * self.cri_fea(fake_fea, real_fea)
        l_g_total += l_g_fea
        
        # G gan + cls loss
        pred_g_fake = self.discriminator(fake_hr)
        #pred_d_real = self.discriminator(hr).detach()
        pred_d_real = self.discriminator(hr)

        l_g_gan = self.l_gan_w * (self.cri_gan(pred_d_real - torch.mean(pred_g_fake), False) +
                                  self.cri_gan(pred_g_fake - torch.mean(pred_d_real), True)) / 2
        l_g_total += l_g_gan

        self.log('val_g_loss', l_g_total, prog_bar=True, on_epoch=True)
        self.log('val_g_pixel_loss', l_g_pix, prog_bar=True, on_epoch=True)
        self.log('val_g_features_loss', l_g_pix, prog_bar=True, on_epoch=True)
        self.log('val_g_adversarial_loss', l_g_gan, prog_bar=True, on_epoch=True)

        if l_g_total < self.best_valid_loss:
            self.best_valid_loss = l_g_total
            self.save_model('best_checkpoint.pth')

    def on_train_end(self):
        self.save_model('last_checkpoint.pth')

    def train_dataloader(self):
      
        transformations = []
        
        if self.hparams.horizontal_flip: 
            transformations.append(RandomHorizontalFlip())
        if self.hparams.vertical_flip:
            transformations.append(RandomVerticalFlip())
        if self.hparams.rotation:
            transformations.append(RandomRotate())

        transformations.append(ToTensor())

        transf = torchvision.transforms.Compose(transformations)

        dataset = EMDataset(self.hparams.patch_size, self.hparams.patch_size, 
                            self.hparams.down_factor, transf=transf, validation=False, 
                            validation_split=self.hparams.validation_split, 
                            hr_imgs_basedir=self.hparams.hr_imgs_basedir, 
                            lr_imgs_basedir=self.hparams.lr_imgs_basedir,
                            only_high_resolution_data=self.hparams.only_high_resolution_data, 
                            only_hr_imgs_basedir=self.hparams.only_hr_images_basedir,
                            type_of_data=self.hparams.type_of_data)

        return DataLoader(dataset, batch_size=self.hparams.batchsize, shuffle=True, num_workers=2)
        
    def val_dataloader(self):
        transf = ToTensor()

        dataset = EMDataset(self.hparams.patch_size, self.hparams.patch_size, 
                            self.hparams.down_factor, transf=transf, validation=True, 
                            validation_split=self.hparams.validation_split, 
                            hr_imgs_basedir=self.hparams.hr_imgs_basedir,
                            lr_imgs_basedir=self.hparams.lr_imgs_basedir,
                            only_high_resolution_data=self.hparams.only_high_resolution_data, 
                            only_hr_imgs_basedir=self.hparams.only_hr_images_basedir,
                            type_of_data=self.hparams.type_of_data)

        return DataLoader(dataset, batch_size=self.hparams.batchsize, shuffle=False)



#--------------------- Display QC Maps --------------------------------

def visualise_image_comparison_QC(image, dimension, Source_folder, Prediction_folder, Ground_truth_folder, QC_folder, QC_scores):
  
  _, image_extension = os.path.splitext(image)

  if image_extension == '.tif':
    img_SSIM_GTvsSource = io.imread(os.path.join(QC_folder, 'SSIM_GTvsSource_'+image))
    img_SSIM_GTvsPrediction = io.imread(os.path.join(QC_folder, 'SSIM_GTvsPrediction_'+image))
    img_RSE_GTvsSource = io.imread(os.path.join(QC_folder, 'RSE_GTvsSource_'+image))
    img_RSE_GTvsPrediction = io.imread(os.path.join(QC_folder, 'RSE_GTvsPrediction_'+image))
  else:
    img_SSIM_GTvsSource = img_as_uint(io.imread(os.path.join(QC_folder, 'SSIM_GTvsSource_'+image)))/65536
    img_SSIM_GTvsPrediction = img_as_uint(io.imread(os.path.join(QC_folder, 'SSIM_GTvsPrediction_'+image)))/65536
    img_RSE_GTvsSource = img_as_uint(io.imread(os.path.join(QC_folder, 'RSE_GTvsSource_'+image)))/65536
    img_RSE_GTvsPrediction = img_as_uint(io.imread(os.path.join(QC_folder, 'RSE_GTvsPrediction_'+image)))/65536

  '''
  SSIM_GTvsP_forDisplay = QC_scores.loc[[image], 'Prediction v. GT mSSIM'].tolist()
  SSIM_GTvsS_forDisplay = QC_scores.loc[[image], 'Input v. GT mSSIM'].tolist()
  NRMSE_GTvsP_forDisplay = QC_scores.loc[[image], 'Prediction v. GT NRMSE'].tolist()
  NRMSE_GTvsS_forDisplay = QC_scores.loc[[image], 'Input v. GT NRMSE'].tolist()
  PSNR_GTvsP_forDisplay = QC_scores.loc[[image], 'Prediction v. GT PSNR'].tolist()
  PSNR_GTvsS_forDisplay = QC_scores.loc[[image], 'Input v. GT PSNR'].tolist()
  '''
 
  SSIM_GTvsP_forDisplay = QC_scores.loc[df['image #'] == image, 'Prediction v. GT mSSIM'].tolist()
  SSIM_GTvsS_forDisplay = QC_scores.loc[df['image #'] == image, 'Input v. GT mSSIM'].tolist()
  NRMSE_GTvsP_forDisplay = QC_scores.loc[df['image #'] == image, 'Prediction v. GT NRMSE'].tolist()
  NRMSE_GTvsS_forDisplay = QC_scores.loc[df['image #'] == image, 'Input v. GT NRMSE'].tolist()
  PSNR_GTvsP_forDisplay = QC_scores.loc[df['image #'] == image, 'Prediction v. GT PSNR'].tolist()
  PSNR_GTvsS_forDisplay = QC_scores.loc[df['image #'] == image, 'Input v. GT PSNR'].tolist()


  plt.figure(figsize=(15,15))

  # Source
  plt.subplot(3,3,1)
  plt.axis('off')
  img_Source = io.imread(os.path.join(Source_QC_folder, image))
  plt.imshow(img_Source,'gray')
  plt.title('Source',fontsize=15)

  # Target (Ground-truth)
  plt.subplot(3,3,2)
  plt.axis('off')
  img_GT = io.imread(os.path.join(Target_QC_folder,image))
  plt.imshow(img_GT ,'gray')
  plt.title('Target',fontsize=15)


  #Prediction
  plt.subplot(3,3,3)
  plt.axis('off')
  #img_Prediction = predictions
  img_Prediction = io.imread(os.path.join(prediction_QC_folder, image))
  plt.imshow(img_Prediction,'gray')
  plt.title('Prediction',fontsize=15)

  #Setting up colours
  cmap = plt.cm.CMRmap

  #SSIM between GT and Source
  plt.subplot(3,3,5)
  #plt.axis('off')
  plt.tick_params(
      axis='both',      # changes apply to the x-axis and y-axis
      which='both',      # both major and minor ticks are affected
      bottom=False,      # ticks along the bottom edge are off
      top=False,        # ticks along the top edge are off
      left=False,       # ticks along the left edge are off
      right=False,         # ticks along the right edge are off
      labelbottom=False,
      labelleft=False)   
  imSSIM_GTvsSource = plt.imshow(img_SSIM_GTvsSource, cmap = cmap, vmin=0, vmax=1)
  plt.colorbar(imSSIM_GTvsSource,fraction=0.046, pad=0.04)
  plt.title('Target vs. Source',fontsize=15)
  plt.xlabel('mSSIM: '+str(round(SSIM_GTvsS_forDisplay[0],3)),fontsize=14)
  plt.ylabel('SSIM maps',fontsize=20, rotation=0, labelpad=75)


  #SSIM between GT and Prediction
  plt.subplot(3,3,6)
  #plt.axis('off')
  plt.tick_params(
      axis='both',      # changes apply to the x-axis and y-axis
      which='both',      # both major and minor ticks are affected
      bottom=False,      # ticks along the bottom edge are off
      top=False,        # ticks along the top edge are off
      left=False,       # ticks along the left edge are off
      right=False,         # ticks along the right edge are off
      labelbottom=False,
      labelleft=False)  
  imSSIM_GTvsPrediction = plt.imshow(img_SSIM_GTvsPrediction, cmap = cmap, vmin=0,vmax=1)
  plt.colorbar(imSSIM_GTvsPrediction,fraction=0.046, pad=0.04)
  plt.title('Target vs. Prediction',fontsize=15)
  plt.xlabel('mSSIM: '+str(round(SSIM_GTvsP_forDisplay[0],3)),fontsize=14)


  #Root Squared Error between GT and Source
  plt.subplot(3,3,8)
  #plt.axis('off')
  plt.tick_params(
      axis='both',      # changes apply to the x-axis and y-axis
      which='both',      # both major and minor ticks are affected
      bottom=False,      # ticks along the bottom edge are off
      top=False,        # ticks along the top edge are off
      left=False,       # ticks along the left edge are off
      right=False,         # ticks along the right edge are off
      labelbottom=False,
      labelleft=False) 
  imRSE_GTvsSource = plt.imshow(img_RSE_GTvsSource, cmap = cmap, vmin=0, vmax = 1)
  plt.colorbar(imRSE_GTvsSource,fraction=0.046,pad=0.04)
  plt.title('Target vs. Source',fontsize=15)
  plt.xlabel('NRMSE: '+str(round(NRMSE_GTvsS_forDisplay[0],3))+', PSNR: '+str(round(PSNR_GTvsS_forDisplay[0],3)),fontsize=14)  
  plt.ylabel('RSE maps',fontsize=20, rotation=0, labelpad=75)

  #Root Squared Error between GT and Prediction
  plt.subplot(3,3,9)
  #plt.axis('off')
  plt.tick_params(
      axis='both',      # changes apply to the x-axis and y-axis
      which='both',      # both major and minor ticks are affected
      bottom=False,      # ticks along the bottom edge are off
      top=False,        # ticks along the top edge are off
      left=False,       # ticks along the left edge are off
      right=False,         # ticks along the right edge are off
      labelbottom=False,
      labelleft=False) 
  imRSE_GTvsPrediction = plt.imshow(img_RSE_GTvsPrediction, cmap = cmap, vmin=0, vmax=1)
  plt.colorbar(imRSE_GTvsPrediction,fraction=0.046,pad=0.04)
  plt.title('Target vs. Prediction',fontsize=15)
  plt.xlabel('NRMSE: '+str(round(NRMSE_GTvsP_forDisplay[0],3))+', PSNR: '+str(round(PSNR_GTvsP_forDisplay[0],3)),fontsize=14)
  #plt.savefig(QC_folder+"/QC_example_data.png",bbox_inches='tight',pad_inches=0)


# **2. Initialise the Colab session**
---







## **2.1. Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

<font size = 4>Go to **Runtime -> Change the Runtime type**

<font size = 4>**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

<font size = 4>**Accelerator: GPU** *(Graphics processing unit)*


In [ ]:
#@markdown ##Run this cell to check if you have GPU access

import tensorflow as tf
if tf.test.gpu_device_name()=='':
  print('You do not have GPU access.') 
  print('Did you change your runtime ?') 
  print('If the runtime setting is correct then Google did not allocate a GPU for your session')
  print('Expect slow performance. To access GPU try reconnecting later')

else:
  print('You have GPU access')
  !nvidia-smi

## **2.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive. 

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Run this cell to connect your Google Drive to Colab

#@markdown * Click on "Connect to Google Drive". 

#@markdown * Sign in your Google Account. 

#@markdown * Click on "Allow". 

#@markdown * Click on `Files` site on the right. Refresh the site. Your Google Drive folder should now be available here as `gdrive`. 

#mounts user's Google Drive to Google Colab.

from google.colab import drive
drive.mount('/content/gdrive')

# **3. Select your paths and parameters**

---


## **3.1. Setting the main training parameters**
---

<font size = 4>The code below allows the user to enter the paths to where the training data is and to define the training parameters. Note that the execution of the cell will take some time as the images from the folders will be read into memory.

<font size = 5> **Paths for training, predictions and results**

<font size = 4>**`Training_source`, `Training_target`:** These are the paths to your folders containing the Training_source and Training_target data respectively. To find the paths of the folders containing the respective datasets, go to your Files on the left of the notebook, navigate to the folder containing your files and copy the path by right-clicking on the folder, **Copy path** and pasting it into the right box below.
  

<font size = 4>**`Only_high_resolution_data`:** In case your dataset only contains high resolution data, he low resolution data will have to be generated artificially by a crappification function. Choose this if your dataset only contains high resolution data. **Default value: False**

<font size = 4>**`Type_of_data`:** In order to generate the low resolution data, it has to be known if the data is from *Electron microscopy* or *Fluorescence*. Based on [Deep Learning-Based Point-Scanning Super-Resolution Imaging](https://www.biorxiv.org/content/10.1101/740548v8) paper, this has to be known to be able of using the suitable crappification function.  **Default value: Electron microscopy**

<font size = 4>**`Down_factor`:** Scaling factor by which every dimension of the HR images is reduced. For example, if an HR image dimension is 256x256, and  its LR conterpart is 128x128, the down_factor is 2. Typical values are 2, 4, etc. This is a **critical parameter** that depends on the acquisition of your LR and HR images. **Default value: 2**

<font size = 4>**`Training_high_resolution_folder`:** If *Only_high_resolution_data* has been selected, previous folder paths will not be operative and the path to the high resolution folder will have to be inserted here.

<font size = 4>**`Model_name`:** Use only my_model -style, not my-model (Use "_" not "-"). Do not use spaces in the name. Avoid using the name of an existing model (saved in the same folder) as it will be overwritten.

<font size = 4>**`Model_path`**: Enter the path where your model will be saved once trained (for instance your result folder).

<font size = 5>**Training parameters**

<font size = 4>**`Number_of_epochs`:** Input how many epochs (rounds) the network will be trained on. Since we use a fast-convergence algorithm, preliminary results can already be observed after 5-10 epochs, and full training could be achieved with as few as 15-20 epochs. Evaluate the  performance after training (see 5.). **Default value: 10**


<font size = 5>**Advanced parameters - experienced users only**

<font size =4>**`Batch_size:`** This parameter defines the number of patches seen in each training step. Reducing or increasing the **batch size** may slow or speed up your training, respectively, and can influence network performance. **Default value: 2**

<font size = 4>**`Percentage_validation`:**  Input the percentage of your training dataset you want to use to validate the network during training. **Default value: 10** 

<font size = 4>**`Generator_initial_learning_rate`:**  Input the initial value to be used as learning rate for the generator network. **Default value: 0.0001**

<font size = 4>**`Discriminator_initial_learning_rate`:**  Input the initial value to be used as learning rate for the discriminator network. **Default value: 0.0001**

<font size = 4>**`Source_patch_size`:** Size in both X and Y axis for the training patches that will be taken from each training image. A lower value would end up in a smaller patch and consequently in a faster training.  **Default value: 64**


In [ ]:
#@markdown ###Path to training images:

Training_source = "" #@param {type:"string"}
Training_target = "" #@param {type:"string"}

#@markdown #####In case you do not have paired LR and HR images, instead of filling the two parameters above completer the ones below: 

Only_high_resolution_data = False #@param {type:"boolean"}
Type_of_data = "Electron microscopy" #@param ["Electron microscopy", "Fluorescence"]
Training_high_resolution_folder = "" #@param {type:"string"}

Down_factor = 4 #@param {type:"number"}
if not Only_high_resolution_data:
    down_factor_x, down_factor_y = calculate_down_factor(Training_source, Training_target)
    if down_factor_x != down_factor_y:
        raise ValueError("Down factors on the width and height of the received images are not the same")
    else:
        Down_factor = down_factor_x
    if Down_factor % 1 !=0:
        raise ValueError("Down factor between hR and LR images is not a whole number.")
    else:
        Down_factor = int(Down_factor)

# Model name and path
#@markdown ###Names of both the model and path to the folder:
Model_name = "" #@param {type:"string"}
Model_path = "" #@param {type:"string"}

# Other parameters for training.
#@markdown ###Training Parameters
#@markdown Number of epochs:

Number_of_epochs =  10#@param {type:"number"}


#@markdown ###Advanced Parameters

Use_Default_Advanced_Parameters = False #@param {type:"boolean"}
#@markdown ###If not, please input:

Batch_size =   2#@param {type:"number"}
Percentage_validation =  10 #@param {type:"number"}
Generator_initial_learning_rate = 0.0001 #@param {type:"number"}
Discriminator_initial_learning_rate = 0.0001 #@param {type:"number"}

Source_patch_size =  64#@param {type:"number"}

if (Use_Default_Advanced_Parameters): 
  print("Default advanced parameters enabled")
  Batch_size = 2
  Percentage_validation = 10
  Generator_initial_learning_rate = 0.0001
  Discriminator_initial_learning_rate = 0.0001
  Source_patch_size = 128

# target_patch_size = source_patch_size * down_factor

#Here we define the percentage to use for validation
val_split = Percentage_validation/100


#here we check that no model with the same name already exist, if so delete
#if os.path.exists(model_path+'/'+model_name):
#  shutil.rmtree(model_path+'/'+model_name)

full_model_path = os.path.join(Model_path, Model_name)
if os.path.exists(full_model_path):
  print(bcolors.WARNING+'!! WARNING: Model folder already exists and will be overwritten if a model is trained !!')
  shutil.rmtree(full_model_path)
  
os.makedirs(full_model_path)
os.makedirs(os.path.join(full_model_path,'Quality Control'))

## **3.2. Data augmentation**
---
<font size = 4>

<font size = 4>Data augmentation can improve training progress by amplifying differences in the dataset. This can be useful if the available dataset is small since, in this case, it is possible that a network could quickly learn every example in the dataset (overfitting), without augmentation. Augmentation is not necessary for training and if your training dataset is large you should disable it.

<font size = 4>Data augmentation is performed here by rotating the patches in XY-Plane and flip them along X-Axis and Y-Axis. This only works if the images are square in XY.


In [ ]:
#@markdown ###Data augmentation
Use_Data_augmentation = True #@param{type:"boolean"}

#@markdown Select this option if you want to use augmentation to increase the size of your dataset

#@markdown **Rotate each image randomly by 90 degrees.**
Rotation = True #@param{type:"boolean"}

#@markdown **Flip each image once around the x and y axis of the stack.**
Horizontal_flip = True #@param{type:"boolean"}
Vertical_flip = True #@param{type:"boolean"}


if Use_Data_augmentation:
  print("Data augmentation enabled")
else:
  print(bcolors.WARNING+"Data augmentation disabled")

## **3.3. Using weights from a pre-trained model as initial weights**
---
<font size = 4>  Here, you can set the the path to a pre-trained model from which the weights can be extracted and used as a starting point for this training session. **This pre-trained model needs to be a WGAN model**. 

<font size = 4> This option allows you to perform training over multiple Colab runtimes or to do transfer learning using models trained outside of ZeroCostDL4Mic. **You do not need to run this section if you want to train a network from scratch**.

<font size = 4> In order to continue training from the point where the pre-trained model left off, it is adviseable to also **load the learning rate** that was used when the training ended. This is automatically saved for models trained with ZeroCostDL4Mic and will be loaded here. If no learning rate can be found in the model folder provided, the default learning rate will be used. 

In [ ]:
# @markdown ##Loading weights from a pre-trained network


Use_pretrained_model = False #@param {type:"boolean"}

Weights_choice = "last" #@param ["last", "best"]

#@markdown ###If you do, please provide the path to the model folder:
Pretrained_model_path = "" #@param {type:"string"}


# Check if we load a previously trained model
if not Use_pretrained_model:
  print(bcolors.WARNING+'No pretrained network will be used.')

else:
  checkpoint_file_path = os.path.join(Pretrained_model_path, Weights_choice+"_checkpoint.pth")

  # Check the model exist
  if not os.path.exists(checkpoint_file_path):
    print(bcolors.WARNING+'WARNING: Pretrained model does not exist.')
    Use_pretrained_model = False
    print(bcolors.WARNING+'No pretrained network will be used.')
  else:
    print("Pretrained model "+os.path.basename(Pretrained_model_path)+" was found and will be loaded prior to training.")


# **4. Train the network**
---

## **4.1. Prepare the data and model for training**
---
<font size = 4>Here, we use the information from section 3 to build the model and convert the training data into a suitable format for training. A pair or LR-HR training images will be displayed at the end of the process.

In [ ]:
#@markdown ##Play this cell to prepare the model for training

if Use_pretrained_model:
  gen_checkpoint = checkpoint_file_path
else:
  gen_checkpoint = None

logger = CSVLogger(full_model_path + '/Quality Control', name='Logger')

lr_monitor = LearningRateMonitor(logging_interval='epoch')
checkpoints = ModelCheckpoint(monitor='val_ssim', mode='max', save_top_k=3, every_n_train_steps=5, save_last=True, filename="{epoch:02d}-{val_ssim:.3f}")

model = ESRGANplus(
    batchsize=Batch_size,
    patch_size=Source_patch_size,
    down_factor=Down_factor,
    learning_rate_g=Generator_initial_learning_rate,
    learning_rate_d=Discriminator_initial_learning_rate,
    n_critic_steps = 5,
    validation_split = val_split,
    epochs = Number_of_epochs,
    rotation = Rotation,
    horizontal_flip = Horizontal_flip,
    vertical_flip = Vertical_flip,
    hr_imgs_basedir = Training_target, 
    lr_imgs_basedir = Training_source,
    only_high_resolution_data = Only_high_resolution_data,
    only_hr_images_basedir = Training_high_resolution_folder,
    type_of_data = Type_of_data,
    save_basedir = full_model_path,
    gen_checkpoint = gen_checkpoint
)

trainer = Trainer(
    gpus=1, 
    max_epochs=Number_of_epochs, 
    logger=logger, 
    callbacks=[checkpoints, lr_monitor]
)

## **4.2. Start Training**
---
<font size = 4>When playing the cell below you should see updates after each epoch (round). Network training can take some time.

<font size = 4>* **CRITICAL NOTE:** Google Colab has a time limit for processing (to prevent using GPU power for datamining). Training time must be less than 12 hours! If training takes longer than 12 hours, please decrease the number of epochs or number of patches.

<font size = 4>Once training is complete, the trained model is automatically saved on your Google Drive, in the **model_path** folder that was selected in Section 3. It is however wise to download the folder as all data can be erased at the next training if using the same folder.

In [ ]:
#@markdown ##Start training
import time
start = time.time()

trainer.fit(model)

dt = time.time() - start
mins, sec = divmod(dt, 60) 
hour, mins = divmod(mins, 60) 
print("Time elapsed:",hour, "hour(s)",mins,"min(s)",round(sec),"sec(s)")

# **5. Evaluate your model**
---

<font size = 4>This section allows the user to perform important quality checks on the validity and generalisability of the trained model. 

<font size = 4>**We highly recommend to perform quality control on all newly trained models.**



In [ ]:
# model name and path
#@markdown ###Do you want to assess the model you just trained ?
Use_the_current_trained_model = True #@param {type:"boolean"}

#@markdown ###If not, please provide the path to model folder and the scale between the low and high resolution images:

QC_model_folder = "" #@param {type:"string"}
QC_down_factor = 2 #@param {type:"number"}

#Here we define the loaded model name and path
QC_model_name = os.path.basename(QC_model_folder)
QC_model_path = os.path.dirname(QC_model_folder)

if Use_the_current_trained_model: 
  QC_model_name = Model_name
  QC_model_path = Model_path
else:
  Down_factor=QC_down_factor

full_QC_model_path = os.path.join(QC_model_path, QC_model_name)

#print(full_QC_model_path)

if os.path.exists(full_QC_model_path):
  print("The "+QC_model_name+" network will be evaluated")
else:
  W  = '\033[0m'  # white (normal)
  R  = '\033[31m' # red
  print(R+'!! WARNING: The chosen model does not exist !!'+W)
  print('Please make sure you provide a valid model path and model name before proceeding further.')


## **5.1. Inspection of the loss function**
---

<font size = 4>First, it is good practice to evaluate the training progress by comparing the training loss with the validation loss. The latter is a metric which shows how well the network performs on a subset of unseen data which is set aside from the training dataset. For more information on this, see for example [this review](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6381354/) by Nichols *et al.*

<font size = 4>**Training loss** describes an error value after each epoch for the difference between the model's prediction and its ground-truth target.

<font size = 4>**Validation loss** describes the same error value between the model's prediction on a validation image and compared to it's target.

<font size = 4>During training both values should decrease before reaching a minimal value which does not decrease further even after more training. Comparing the development of the validation loss with the training loss can give insights into the model's performance.

<font size = 4>Decreasing **Training loss** and **Validation loss** indicates that training is still necessary and increasing the `number_of_epochs` is recommended. Note that the curves can look flat towards the right side, just because of the y-axis scaling. The network has reached convergence once the curves flatten out. After this point no further training is required. If the **Validation loss** suddenly increases again an the **Training loss** simultaneously goes towards zero, it means that the network is overfitting to the training data. In other words the network is remembering the exact patterns from the training data and no longer generalizes well to unseen data. In this case the training dataset has to be increased.

In [ ]:
#@markdown ##Play the cell to show the plots of training errors vs. epoch number

logger_path = os.path.join(full_QC_model_path + '/Quality Control/Logger')
all_logger_versions = [os.path.join(logger_path, dname) for dname in os.listdir(logger_path)]
last_logger = all_logger_versions[-1]

train_csv_path = last_logger + '/metrics.csv'

if not os.path.exists(train_csv_path):
  print('The path does not contain a csv file containing the loss and validation evolution of the model')
else:
  with open(train_csv_path,'r') as csvfile:
    csvRead = csv.reader(csvfile, delimiter=',')
    keys = next(csvRead)
    keys.remove('step')
    train_metrics = {'g_lr':[], 'd_lr':[],
                    'g_loss_step':[], 'g_pixel_loss_step':[], 
                    'g_features_loss_step':[], 'g_adversarial_loss_step': [],
                    'd_loss_step':[], 'd_real_step':[], 'd_fake_step':[],
                    'epoch':[],
                    'val_ssim':[], 'val_psnr':[],
                    'val_g_loss':[], 'val_g_pixel_loss':[], 
                    'val_g_features_loss':[], 'val_g_adversarial_loss': [],
                    'g_loss_epoch':[], 'g_pixel_loss_epoch':[], 
                    'g_features_loss_epoch':[], 'g_adversarial_loss_epoch': [],
                    'd_loss_epoch':[], 'd_real_epoch':[], 'd_fake_epoch':[]
                     }

    for row in csvRead:
      step = int(row[2])
      row.pop(2)
      for i, row_value in enumerate(row):
        if row_value:
          train_metrics[keys[i]].append([step, float(row_value)])


  epochNumber = range(len(train_metrics['g_loss_epoch']))
  plt.figure(figsize=(20,30))

  plt.subplot(4,1,1)
  plt.plot(epochNumber,[e[1] for e in train_metrics['g_loss_epoch']], label='Training loss')
  plt.plot(epochNumber,[e[1] for e in train_metrics['val_g_loss']], label='Validation loss')
  plt.title('Generator\'s total training and validation loss VS epochs')
  plt.ylabel('Generator loss')
  plt.xlabel('Epoch number')
  plt.legend()

  plt.subplot(4,1,2)
  plt.plot(epochNumber,[e[1] for e in train_metrics['g_pixel_loss_epoch']], label='Training loss')
  plt.plot(epochNumber,[e[1] for e in train_metrics['val_g_pixel_loss']], label='Validation loss')
  plt.title('Generator\'s pixel training and validation loss VS epochs')
  plt.ylabel('Pixel loss')
  plt.xlabel('Epoch number')
  plt.legend()

  plt.subplot(4,1,3)
  plt.plot(epochNumber,[e[1] for e in train_metrics['g_features_loss_epoch']], label='Training loss')
  plt.plot(epochNumber,[e[1] for e in train_metrics['val_g_features_loss']], label='Validation loss')
  plt.title('Generator\'s features training and validation loss VS epochs')
  plt.ylabel('Features loss')
  plt.xlabel('Epoch number')
  plt.legend()

  plt.subplot(4,1,4)
  plt.plot(epochNumber,[e[1] for e in train_metrics['g_adversarial_loss_epoch']], label='Training loss')
  plt.plot(epochNumber,[e[1] for e in train_metrics['val_g_adversarial_loss']], label='Validation loss')
  plt.title('Generator\'s adversarial training and validation loss VS epochs')
  plt.ylabel('Generator\'s adversarial loss')
  plt.xlabel('Epoch number')
  plt.legend()

  plt.savefig(QC_model_path+'/'+QC_model_name+'/Quality Control/TrainVal_epoch_losses_plots.png')
  plt.show()


In [ ]:
#@markdown ##Play the cell to show the plots of training errors vs. steps number

plt.figure(figsize=(20,30))
  
plt.subplot(7,1,1)
plt.plot([e[0] for e in train_metrics['g_loss_step']], [e[1] for e in train_metrics['g_loss_step']], label='Training loss')
plt.title('Generator\'s total loss during training steps')
plt.ylabel('Generator total loss')
plt.xlabel('Step number')

plt.subplot(7,1,2)
plt.plot([e[0] for e in train_metrics['g_pixel_loss_step']], [e[1] for e in train_metrics['g_pixel_loss_step']], label='Training loss')
plt.title('Generator\'s pixel loss during training steps')
plt.ylabel('Pixel loss')
plt.xlabel('Step number')

plt.subplot(7,1,3)
plt.plot([e[0] for e in train_metrics['g_features_loss_step']], [e[1] for e in train_metrics['g_features_loss_step']], label='Training loss')
plt.title('Generator\'s feature loss during training steps')
plt.ylabel('Feature loss')
plt.xlabel('Step number')

plt.subplot(7,1,4)
plt.plot([e[0] for e in train_metrics['g_adversarial_loss_step']], [e[1] for e in train_metrics['g_adversarial_loss_step']], label='Training loss')
plt.title('Generator\'s adversarial loss during training steps')
plt.ylabel('Generator\'s adversarial loss')
plt.xlabel('Step number')

plt.subplot(7,1,5)
plt.plot([e[0] for e in train_metrics['d_loss_step']], [e[1] for e in train_metrics['d_loss_step']], label='Training loss')
plt.title('Discriminator\'s total loss during training steps')
plt.ylabel('Discriminator\'s loss')
plt.xlabel('Step number')

plt.subplot(7,1,6)
plt.plot([e[0] for e in train_metrics['d_real_step']], [e[1] for e in train_metrics['d_real_step']], label='Training loss')
plt.title('Discriminator\'s real loss during training steps')
plt.ylabel('Discriminator\'s real loss')
plt.xlabel('Step number')

plt.subplot(7,1,7)
plt.plot([e[0] for e in train_metrics['d_fake_step']], [e[1] for e in train_metrics['d_fake_step']])
plt.title('Discriminator\'s fake loss during training steps')
plt.ylabel('Discriminator\'s fake loss')
plt.xlabel('Step number')

plt.savefig(QC_model_path+'/'+QC_model_name+'/Quality Control/Train_steps_losses_plots.png')
plt.show()

## **5.2. Image predictions**
---


In [ ]:
#@markdown ##Choose the folders that contain your Quality Control dataset
Source_QC_folder = "" #@param{type:"string"}
Target_QC_folder = "" #@param{type:"string"}


#@markdown #####In case you do not have paired LR and HR images, instead of filling the two parameters above completer the ones below: 

Only_high_resolution_data = True #@param {type:"boolean"}
Type_of_data = "Electron microscopy" #@param ["Electron microscopy", "Fluorescence"] 
QC_high_resolution_folder = "/content/gdrive/MyDrive/TFM - dataset/test" #@param {type:"string"}


# Create a list of sources

# Insert code to perform predictions on all datasets in the Source_QC folder
if Only_high_resolution_data:
  Source_QC_folder = "/content/LR_images"
  Target_QC_folder = QC_high_resolution_folder

_, test_extension = os.path.splitext(os.listdir(Target_QC_folder)[0])
test_filenames = [x for x in os.listdir( Target_QC_folder ) if x.endswith(test_extension)]
test_filenames.sort()

print('Available images : ' + str(len(test_filenames)))

model = ESRGANplus(gen_checkpoint = full_QC_model_path + '/best_checkpoint.pth')


trainer = Trainer(gpus=1)

dataset = EMDataset(1, 1, Down_factor, transf=ToTensor(), 
                    validation=False, validation_split=0.001, 
                    hr_imgs_basedir=Target_QC_folder, 
                    lr_imgs_basedir=Source_QC_folder,
                    only_high_resolution_data=Only_high_resolution_data, 
                    only_hr_imgs_basedir=QC_high_resolution_folder,
                    type_of_data=Type_of_data)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
predictions = trainer.predict(model, dataloaders=dataloader)

X_test, Y_test = create_complete_images( Source_QC_folder, Target_QC_folder, 
                                        Only_high_resolution_data, 
                                        Type_of_data, 
                                        QC_high_resolution_folder, 
                                        Down_factor)

X_test = np.expand_dims(X_test, axis=-1)
Y_test = np.expand_dims(Y_test, axis=-1)

if Only_high_resolution_data:
  Source_QC_folder = "/content/LR_images"
  Target_QC_folder = QC_high_resolution_folder

_, test_extension = os.path.splitext(os.listdir(Target_QC_folder)[0])

test_filenames = [x for x in os.listdir( Target_QC_folder ) if x.endswith(test_extension)]
test_filenames.sort()

print( 'Available images : ' + str( len(test_filenames)) )

if Only_high_resolution_data:
  if os.path.exists(Source_QC_folder):
    shutil.rmtree(Source_QC_folder)
  os.makedirs(Source_QC_folder)
  for i, image in enumerate(np.expand_dims(X_test, axis=-1)):
    tf.keras.preprocessing.image.save_img(Source_QC_folder+'/'+test_filenames[i], image, data_format=None, file_format=None)

#Save the predictions
prediction_QC_folder = os.path.join(full_QC_model_path, 'Quality Control', 'Prediction')
if os.path.exists(prediction_QC_folder):
  shutil.rmtree(prediction_QC_folder)
os.makedirs(prediction_QC_folder)

_MSSSIM_WEIGHTS = (0.0448, 0.2856, 0.3001, 0.2363, 0.1333)

psnr_array = []
ssim_array = []
mssim_array = []

for i, pred in enumerate(predictions):
  pred = np.expand_dims(np.squeeze(pred.detach().numpy()),axis=-1)

  tf.keras.preprocessing.image.save_img(prediction_QC_folder+'/'+test_filenames[i], 
                                        pred, data_format=None, file_format=None)

  pred = np.clip( pred, a_min=0, a_max=1 )

  hr_image = Y_test[i]

  psnr_array.append(metrics.peak_signal_noise_ratio(hr_image[:,:,0], pred[:,:,0]))
  ssim_array.append(metrics.structural_similarity(hr_image[:,:,0], pred[:,:,0]))
  mssim_array.append(mssim(hr_image.astype('float32'), pred, max_val=1, 
                            power_factors=_MSSSIM_WEIGHTS, filter_size=11,
                            filter_sigma=1.5, k1=0.01, k2=0.03))

psnr_mean = np.mean(psnr_array)
ssim_mean = np.mean(ssim_array)
mssim_mean = np.mean(mssim_array)

del predictions, X_test, Y_test, model, trainer, dataset, dataloader

# ------------- For display ------------
print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
print("Choose the image file (Once selected, the new set of images will take a few seconds to appear):")
print(" ")
@interact
def show_prediction_results(file = os.listdir(prediction_QC_folder)):
  imageLR = io.imread(os.path.join(Source_QC_folder, file))
  imageHR = io.imread(os.path.join(Target_QC_folder, file))
  imageP = io.imread(os.path.join(prediction_QC_folder, file))

  plt.figure(figsize=(25,25))
  plt.subplot(3, 3, 1)
  plt.imshow( imageLR, 'gray' )
  plt.title( 'Low resolution' )
  # Side by side with its "ground truth"
  plt.subplot(3, 3, 2)
  plt.imshow( imageHR, 'gray' )
  plt.title( 'High resolution' )
  # ant its prediction
  plt.subplot(3, 3, 3)
  plt.imshow( imageP, 'gray' )
  plt.title( 'Prediction' ) 

print('------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')


## **5.3. Error mapping and quality metrics estimation**
---


<font size = 4>This section will calculate the SSIM, PNSR and MSSIM metrics between the predicted and target images to evaluate the quality of the results.

<font size = 4>**1. The SSIM (structural similarity) map** 

<font size = 4>The SSIM metric is used to evaluate whether two images contain the same structures. It is a normalized metric and an SSIM of 1 indicates a perfect similarity between two images. Therefore for SSIM, the closer to 1, the better.
<font size=4>**mSSIM** is the SSIM value calculated across the entire window of both images.

<font size = 4>**2. PSNR (Peak signal-to-noise ratio)** is a metric that gives the difference between the ground truth and prediction (or source input) in decibels, using the peak pixel values of the prediction and the MSE between the images. The higher the score the better the agreement.





In [ ]:
#@markdown ##Run to calculate the PSNR, SSIM and MSSIM metrics

print("PSNR: ", psnr_mean)
print("SSIM: ", ssim_mean)
print("MSSIM:", mssim_mean)

<font size = 4>The metrics shown in the previous cell are an average of all the test dataset. In the following cell the metrics refer only to the displayed image. You can see all the metrics for each image in a csv file saved in the Quality Control folder.

In [ ]:
#@markdown ##SSIM and RSE map
#@markdown Now we will show the SSIM and RSE maps between the original images (upsampled using simple interpolation) and the target images, together with the maps between the predicted and target images.

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity

def ssim(img1, img2):
  return structural_similarity(img1,img2,data_range=1.,full=True, gaussian_weights=True, use_sample_covariance=False, sigma=1.5)


def normalize(x, pmin=3, pmax=99.8, axis=None, clip=False, eps=1e-20, dtype=np.float32):
    """This function is adapted from Martin Weigert"""
    """Percentile-based image normalization."""

    mi = np.percentile(x,pmin,axis=axis,keepdims=True)
    ma = np.percentile(x,pmax,axis=axis,keepdims=True)
    return normalize_mi_ma(x, mi, ma, clip=clip, eps=eps, dtype=dtype)


def normalize_mi_ma(x, mi, ma, clip=False, eps=1e-20, dtype=np.float32):#dtype=np.float32
    """This function is adapted from Martin Weigert"""
    if dtype is not None:
        x   = x.astype(dtype,copy=False)
        mi  = dtype(mi) if np.isscalar(mi) else mi.astype(dtype,copy=False)
        ma  = dtype(ma) if np.isscalar(ma) else ma.astype(dtype,copy=False)
        eps = dtype(eps)

    try:
        import numexpr
        x = numexpr.evaluate("(x - mi) / ( ma - mi + eps )")
    except ImportError:
        x =(x - mi) / ( ma - mi + eps )

    if clip:
        x = np.clip(x,0,1)

    return x

def norm_minmse(gt, x, normalize_gt=True):
    """This function is adapted from Martin Weigert"""

    """
    normalizes and affinely scales an image pair such that the MSE is minimized  
     
    Parameters
    ----------
    gt: ndarray
        the ground truth image      
    x: ndarray
        the image that will be affinely scaled 
    normalize_gt: bool
        set to True of gt image should be normalized (default)
    Returns
    -------
    gt_scaled, x_scaled 
    """
    if normalize_gt:
        gt = normalize(gt, 0.1, 99.9, clip=False).astype(np.float32, copy = False)
    x = x.astype(np.float32, copy=False) - np.mean(x)
    #x = x - np.mean(x)
    gt = gt.astype(np.float32, copy=False) - np.mean(gt)
    #gt = gt - np.mean(gt)
    x_scaled = transform.resize( x, (gt.shape[0], gt.shape[1]), order=1 )
    
    scale = np.cov(x_scaled.flatten(), gt.flatten())[0, 1] / np.var(x_scaled.flatten())
    return gt, scale * x_scaled
    #return gt, x_scaled


# Open and create the csv file that will contain all the QC metrics
with open(QC_model_path+"/"+QC_model_name+"/Quality Control/QC_metrics_"+QC_model_name+".csv", "w", newline='') as file:
    writer = csv.writer(file)

    # Write the header in the csv file
    writer.writerow(["image #","Prediction v. GT mSSIM","Input v. GT mSSIM", "Prediction v. GT NRMSE", "Input v. GT NRMSE", "Prediction v. GT PSNR", "Input v. GT PSNR"])  

    # Let's loop through the provided dataset in the QC folders
    print('Computing maps...')

    _, image_extension = os.path.splitext(os.listdir(Target_QC_folder)[0])

    for i in os.listdir(Target_QC_folder):
      if not os.path.isdir(os.path.join(Source_QC_folder,i)) and i.endswith(image_extension):

      # -------------------------------- Target test data (Ground truth) --------------------------------
        test_GT = io.imread(os.path.join(Target_QC_folder, i))
        #test_GT = test_patches_gt

      # -------------------------------- Source test data --------------------------------
        test_source = io.imread(os.path.join(Source_QC_folder,i))
        #test_source = test_patches_wf

      # Normalize the images wrt each other by minimizing the MSE between GT and Source image
        test_GT_norm,test_source_norm = norm_minmse(test_GT, test_source, normalize_gt=True)

      # -------------------------------- Prediction --------------------------------
        test_prediction = io.imread(os.path.join(prediction_QC_folder,i))

      # Normalize the images wrt each other by minimizing the MSE between GT and prediction
        test_GT_norm,test_prediction_norm = norm_minmse(test_GT, test_prediction, normalize_gt=True)        

      # -------------------------------- Calculate the metric maps and save them --------------------------------

      # Calculate the SSIM maps
        index_SSIM_GTvsPrediction, img_SSIM_GTvsPrediction = ssim(test_GT_norm, test_prediction_norm)
        index_SSIM_GTvsSource, img_SSIM_GTvsSource = ssim(test_GT_norm, test_source_norm)

      #Save ssim_maps
        img_SSIM_GTvsPrediction_32bit = np.float32(img_SSIM_GTvsPrediction)
        img_SSIM_GTvsSource_32bit = np.float32(img_SSIM_GTvsSource)
        if image_extension == '.tif':
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/SSIM_GTvsPrediction_'+i, img_SSIM_GTvsPrediction_32bit)
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/SSIM_GTvsSource_'+i, img_SSIM_GTvsSource_32bit)
        else:
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/SSIM_GTvsPrediction_'+i, img_as_uint(img_SSIM_GTvsPrediction_32bit))
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/SSIM_GTvsSource_'+i, img_as_uint(img_SSIM_GTvsSource_32bit))
            
      # Calculate the Root Squared Error (RSE) maps
        img_RSE_GTvsPrediction = np.sqrt(np.square(test_GT_norm - test_prediction_norm))
        img_RSE_GTvsSource = np.sqrt(np.square(test_GT_norm - test_source_norm))

      # Save SE maps
        img_RSE_GTvsPrediction_32bit = np.float32(img_RSE_GTvsPrediction)
        img_RSE_GTvsSource_32bit = np.float32(img_RSE_GTvsSource)
        if image_extension == '.tif':
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/RSE_GTvsPrediction_'+i, np.clip(img_RSE_GTvsPrediction_32bit, -1, 1))
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/RSE_GTvsSource_'+i, np.clip(img_RSE_GTvsSource_32bit, -1, 1))
        else:
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/RSE_GTvsPrediction_'+i, img_as_uint(np.clip(img_RSE_GTvsPrediction_32bit, -1, 1)))
          io.imsave(QC_model_path+'/'+QC_model_name+'/Quality Control/RSE_GTvsSource_'+i, img_as_uint(np.clip(img_RSE_GTvsSource_32bit, -1, 1)))


      # -------------------------------- Calculate the RSE metrics and save them --------------------------------

      # Normalised Root Mean Squared Error (here it's valid to take the mean of the image)
        NRMSE_GTvsPrediction = np.sqrt(np.mean(img_RSE_GTvsPrediction))
        NRMSE_GTvsSource = np.sqrt(np.mean(img_RSE_GTvsSource))
        
      # We can also measure the peak signal to noise ratio between the images
        PSNR_GTvsPrediction = psnr(test_GT_norm,test_prediction_norm,data_range=1.0)
        PSNR_GTvsSource = psnr(test_GT_norm,test_source_norm,data_range=1.0)

        writer.writerow([i,str(index_SSIM_GTvsPrediction),str(index_SSIM_GTvsSource),str(NRMSE_GTvsPrediction),str(NRMSE_GTvsSource),
                         str(PSNR_GTvsPrediction),str(PSNR_GTvsSource)])

# All data is now processed saved
Test_FileList = os.listdir(prediction_QC_folder) # this assumes, as it should, that both source and target are named the same


plt.figure(figsize=(15,15))
# Currently only displays the last computed set, from memory

df = pd.read_csv(QC_model_path+"/"+QC_model_name+"/Quality Control/QC_metrics_"+QC_model_name+".csv")

# ------------- For display ------------
print('------------------------------------------------------------------------------------------------------------------------')
print("Choose the image file (once selected, the new set of images will take a few seconds to appear):")
print(" ")
@interact
def show_QC_results(file = os.listdir(Source_QC_folder)):
  visualise_image_comparison_QC(image = file, dimension='2D', Source_folder=Source_QC_folder , Prediction_folder= prediction_QC_folder, Ground_truth_folder=Target_QC_folder, QC_folder=QC_model_path+"/"+QC_model_name+"/Quality Control", QC_scores= df )  

print('------------------------------------------------------------------------------------------------------------------------')

# **6. Using the trained model**

---

<font size = 4>In this section the unseen data is processed using the trained model (in section 4). First, your unseen images are uploaded and prepared for prediction. After that your trained model from section 4 is activated and finally saved into your Google Drive.

## **6.1. Generate prediction(s) from unseen dataset**
---
<font size = 4>The current trained model (from section 4.2) can now be used to process images. If you want to use an older model, untick the **Use_the_current_trained_model** box and enter the name and path of the model to use. Predicted output images are saved in your **Result_folder** folder as restored image stacks (ImageJ-compatible TIFF images).

<font size = 4>**`Data_folder`:** This folder should contain the images that you want to use your trained network on for processing.

<font size = 4>**`Result_folder`:** This folder will contain the predicted output images.

In [ ]:
#@markdown ### Provide the path to your dataset and to the folder where the predictions are saved, then play the cell to predict outputs from your unseen images.

Data_folder = "" #@param {type:"string"}
Result_folder = "" #@param {type:"string"}


#@markdown ###Does **Data_folder** contain *High resolution* images?
#@markdown ##### If your data only contains *High resolution* images, check the option below. The *Low resolution* images will be generated from the *High resolution* data given in the **Data_foder**.    

Are_high_resolution_images = False #@param {type:"boolean"}

Type_of_data = "Electron microscopy" #@param ["Electron microscopy", "Fluorescence"] 


# model name and path
#@markdown ###Do you want to use the current trained model?
Use_the_current_trained_model = True #@param {type:"boolean"}

#@markdown ###If not, provide the name of the model and path to model folder:
#@markdown #####During training, the model files are automatically saved inside a folder named after model_name in section 3. Provide the path to this folder below.
Prediction_model_folder = "" #@param {type:"string"}
Prediction_down_factor = 2 #@param {type:"number"}


if not os.path.exists(Result_folder):
  print("The "+Result_folder+ " path does not exist")
  os.makedirs(Result_folder)

#Here we find the loaded model name and parent path
Prediction_model_name = os.path.basename(Prediction_model_folder)
Prediction_model_path = os.path.dirname(Prediction_model_folder)

if (Use_the_current_trained_model): 
  print("Using current trained network")
  Prediction_model_name = Model_name
  Prediction_model_path = Model_path
else:
  Down_factor = Prediction_down_factor

full_Prediction_model_path = os.path.join(Prediction_model_path, Prediction_model_name)
if os.path.exists(full_Prediction_model_path):
  print("The "+Prediction_model_name+" network will be used.")
else:
  W  = '\033[0m'  # white (normal)
  R  = '\033[31m' # red
  print(R+'!! WARNING: The chosen model does not exist !!'+W)
  print('Please make sure you provide a valid model path and model name before proceeding further.')

# Read the list of file names


_, extension = os.path.splitext(os.listdir(Data_folder)[0])

filenames = [x for x in os.listdir( Data_folder ) if x.endswith(extension)]
filenames.sort()
print( 'Available images: ' + str( len(filenames)) )


model = ESRGANplus(
    gen_checkpoint = full_QC_model_path + '/best_checkpoint.pth'
)

gen = model.generator

del model

#model.cuda()


#Save the predictions
prediction_QC_folder = os.path.join(Result_folder, 'Quality Control', 'Prediction')
if os.path.exists(Result_folder):
  shutil.rmtree(Result_folder)
os.makedirs(Result_folder)

if Save_generated_low_resolution_images:
    if not os.path.exists(Low_resolution_folder):
      print("The "+Low_resolution_folder+ " path does not exist, it will be generated.")
      os.makedirs(Low_resolution_folder)

print("Predicting...")

for i in tqdm(filenames):
  image = img_as_ubyte( io.imread( Data_folder + '/' + i ) )

  if Are_high_resolution_images:
    if Type_of_data == "Electron microscopy":
      image = em_crappify(image, Down_factor)
    else:
      image = fluo_crappify(image, Down_factor)

  if Save_generated_low_resolution_images:
      tf.keras.preprocessing.image.save_img(Low_resolution_folder+'/'+filenames[i], image, data_format=None, file_format=None)

  image = image/255 # normalize between 0 and 1
  image = np.expand_dims(image, axis=-1)
  image = image.astype(np.float32)
  image = image.transpose((2, 0, 1))
  image = torch.from_numpy(image)

  prediction = gen(image)
  prediction = np.expand_dims(np.squeeze(prediction.detach().numpy()),axis=-1)


  tf.keras.preprocessing.image.save_img(Result_folder+'/'+i, prediction)
  del image
  del prediction

print("Images saved into folder:", Result_folder)

## **6.2. Inspect the predicted output**
---



In [ ]:
# @markdown  ##Run this cell to see some inputs with their corresponding outputs
##Run this cell to display a randomly chosen input and its corresponding predicted output.

# ------------- For display ------------
print('---------------------------------------------------------------------------------------------------------------------------------------------------')
print("Choose the image file (Once selected, the new set of images will take a few seconds to appear):")
print(" ")
@interact
def show_prediction_results(file = sorted(os.listdir(Data_folder))):
  imageHR = io.imread(os.path.join(Data_folder, file))
  imageP = io.imread(os.path.join(Result_folder, file))

  plt.figure(figsize=(25,25))
  plt.subplot(3, 3, 1)
  plt.imshow( imageHR, 'gray' )
  if Are_high_resolution_images:
    plt.title( 'High resolution' )
  else:
    plt.title( 'Low resolution' )
  
  # ant its prediction
  plt.subplot(3, 3, 2)
  if Save_generated_low_resolution_images:
    imageLR = io.imread(os.path.join(Low_resolution_folder, file))
    plt.imshow( imageLR, 'gray' )
    plt.title( 'Low resolution' ) 
    plt.subplot(3, 3, 3)
    plt.imshow( imageP, 'gray' )
    plt.title( 'Prediction' ) 
  else:
    plt.imshow( imageP, 'gray' )
    plt.title( 'Prediction' ) 

print('---------------------------------------------------------------------------------------------------------------------------------------------------')


## **6.3. Download your predictions**
---

<font size = 4>**Store your data** and ALL its results elsewhere by downloading it from Google Drive and after that clean the original folder tree (datasets, results, trained model etc.) if you plan to train or use new networks. Please note that the notebook will otherwise **OVERWRITE** all files which have the same name.

# **7. Version log**
---




#**Thank you for using WGAN!**